# CollagenTransformer: End-to-end transformer model to predict thermal stability of collagen triple helices using an NLP approach 

#### Eesha Khare1,2, Constancio Gonzalez Obeso3, David L. Kaplan3 Markus J. Buehler1,4*

1 Laboratory for Atomistic and Molecular Mechanics (LAMM), Massachusetts Institute of Technology, 77 Massachusetts Ave., Cambridge, MA 02139, USA
2 Department of Materials Science and Engineering, Massachusetts Institute of Technology, 77 Massachusetts Ave., Cambridge, MA 02139, USA
3 Tufts University, Medford, MA, USA
4 Center for Computational Science and Engineering, Schwarzman College of Computing, Massachusetts Institute of Technology, 77 Massachusetts Ave., Cambridge, MA 02139, USA

*mbuehler@mit.edu 

## Collagen transformer model to predict Tm (small transformer model)

In [1]:
import numpy as np
import pandas as pd
import os,sys
import math
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import matplotlib.pyplot as plt
import tqdm

import cv2
import PIL 
from PIL import Image, ImageOps

import pickle
import torchvision.utils as vutils
import torch
 
import torchvision
 
import matplotlib.pyplot as plt
import numpy as np
 
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR, StepLR

print("Torch version:", torch.__version__) 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from einops import rearrange, repeat

import seaborn as sns
import pickle

C:\Users\marku\anaconda3\envs\Huggingface New\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.11.0+cu113


In [3]:
from math import pi, log
import time
import seaborn as sns
from sklearn.metrics import r2_score

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler, MinMaxScaler

In [4]:
!wget https://www.dropbox.com/s/8oiyjvgn3nofkkh/model_IO_REGRESS_1290.pth?dl=0 -O model_IO_REGRESS_1290.pth
!wget https://www.dropbox.com/s/60ximulejl6fq4b/tokenizer_and_qt.pickle?dl=0 -O tokenizer_and_qt.pickle

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
#source: https://github.com/lucidrains/x-transformers
#!pip install x_transformers==0.28.0

from x_transformers import XTransformer,ContinuousTransformerWrapper, Decoder, Encoder

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")


In [7]:
from torch.utils.data import DataLoader,Dataset
from torchvision.io import read_image
import pandas as pd
from sklearn.model_selection import train_test_split

from PIL import Image

to_pil = transforms.ToPILImage()

In [9]:
num_tokens_=25 
max_length = 64

In [10]:
embed_dim_x = 8  # for x
embed_dim = 8 #for position 
embed_dim_conv=8  #for conv
kernel_siz = 3

latent_dim_ =32

nconv=2 #number of convs for input

class MyTotal_pos(nn.Module):
    def __init__(self ):
        super(MyTotal_pos, self).__init__()
        
        self.mSig = nn.Sigmoid()
        self.queries_dim_=embed_dim  + embed_dim_x #dim_
         
         
        
        self.emb_data = nn.Embedding(num_tokens_, embed_dim_x*1,  padding_idx=0)
        self.pos_emb_x = nn.Embedding(max_length, embed_dim*1)
        
        self.fc_x1 = nn.Linear(   embed_dim_x, embed_dim_x )  # INPUT DIM (last), OUTPUT DIM, last
        
        self.fc1 = nn.Linear( max_length,  1)  # INPUT DIM (last), OUTPUT DIM, last
        self.fc2 = nn.Linear( max_length,  1)  # INPUT DIM (last), OUTPUT DIM, last
        self.fc3 = nn.Linear( max_length,  1)  # INPUT DIM (last), OUTPUT DIM, last
        
        self.BatchNorm_1 = nn.BatchNorm1d(max_length)
        self.fc_last = nn.Linear( self.queries_dim_,  1)  
        self.fc_last_2 = nn.Linear( max_length,  1)  
        
        self.convs=[]
        self.skips_cons=[]
        icgg=0
        for _ in range(nconv):
            if icgg==0: #first one opertes on 3 channels only
                self.convs.append(nn.Conv1d(in_channels=embed_dim_x, out_channels=embed_dim_conv, kernel_size=3, stride=1, \
                                        padding='same'))#.cuda())
                 
            if icgg>0:
                self.convs.append(nn.Conv1d(in_channels=embed_dim_x, out_channels=embed_dim_conv, kernel_size=3, stride=1, \
                                        padding='same'))#.cuda())
            icgg=icgg+1
             
        self.convs = nn.ModuleList(self.convs)
        
        self.pos_matrix_i = torch.zeros (max_length, dtype=torch.long)
        
        for i in range (max_length):
            
                self.pos_matrix_i [i]=i
            
 
        self.modelB = ContinuousTransformerWrapper(
            dim_in = self.queries_dim_,
            dim_out = self.queries_dim_,
            max_seq_len = max_length,
            attn_layers = Encoder(
                dim = self.queries_dim_,
                depth = 3,
               heads = 8
               ))
        
    def forward(self, x):
        device = x.device
        n=self.queries_dim_
         
        x = self.emb_data( x)
       
        x = torch.squeeze(x, 1)
        
        x= self.BatchNorm_1(x)
        
        pos_matrix_i_=self.pos_matrix_i.repeat(x.shape[0], 1, 1).to(device=device) 
        
        
        pos_emb_x = self.pos_emb_x( pos_matrix_i_)
        pos_emb_x = torch.squeeze(pos_emb_x, 1)
        
        self.skips_cons=[]
         
        convcount=0
        for  encoder in self.convs:
            if convcount==0:
                x_cc=torch.permute(x, (0,2,1)  )
            else:
                x_cc=torch.permute(x_cc, (0,2,1)  )
            
            x_cc =  encoder.to(device=device)(x_cc)
            x_cc=torch.permute(x_cc, (0,2,1)  )
           
            self.skips_cons.append(x_cc)
            convcount=  convcount+1
            
        
        for skip in self.skips_cons[:-1]:
            x_cc = torch.cat((x_cc, skip), axis=2)

        
        x= torch.cat( (x,    pos_emb_x), 2)
             
        x2 = self.modelB(x) #, queries = query1)
        
        x2=self.fc_last (x2)
        x2=torch.permute(x2, (0,2,1)  )
        x2=self.fc_last_2 (x2)
        #x2=self.mSig(x2)
        
        return x2
 
model = MyTotal_pos(  )

In [11]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

print ("Total parameters: ", pytorch_total_params," trainable parameters: ", pytorch_total_params_trainable)

Total parameters:  108069  trainable parameters:  108069


In [12]:
# loading
with open('tokenizer_and_qt.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    qt = pickle.load(handle)
    

### General inference for a given sequence

In [13]:
from tensorflow.keras.preprocessing import text, sequence
def SS_predictor(model,tokenizer,seq,max_length, qt=None):
    model.eval()
    prot_1 = tokenizer.texts_to_sequences([seq[:]])
    prot_1 = sequence.pad_sequences(prot_1, maxlen=max_length, padding='post' )
    prot_1 = torch.from_numpy(prot_1).to(device)
    
    #print(prot_1.shape)
    prot_tm=model(prot_1) 
    if qt : #inverse scale if PRESENT
        temp_=prot_tm.cpu().detach().numpy() 
        temp_=np.squeeze(temp_, 2)
        prot_tm=qt.inverse_transform(temp_)
    
    prot_tm=np.squeeze(prot_tm)
    
    return prot_tm 

In [14]:
#load model if not loaded yet
name='./model_IO_REGRESS_1290.pth'
model = torch.load(name).to(device)

In [15]:
Tm=SS_predictor (model, tokenizer, 'GPOGPOGPOGPOGPQGPGGPPGPOGPOGPOGPOGPO', max_length, qt)
print (Tm)

62.49929


In [16]:
Tm=SS_predictor (model, tokenizer, 'GPOGPOGPODPOGPOGPOGPOGPO', max_length, qt)
print (Tm)

-6.225907
